Objective is to use census and epa keys to combine population data. The census will provide the population
and the EPA will provide the air quality data.
I would like to break the population (total population, population density) down by counties.

In [9]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
import gmaps

# EPA API Key
from epakey import b_epa_key

# Census & gmaps API Keys
from censuskey import (b_census_key)
c = Census(b_census_key, year=2013)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [ ]:
# Run Census Search to retrieve data on all states
# What we will ultimately want is bringing in population and population density. Latitudes and longitudes
# will have to be verified across datasets to ensure proper merging.
# The c.acs5.get is from the census wrapping in a github repo
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population", "B19013_001E": "Household Income",
                                      "NAME": "Name", "state": "State"})



census_pd.head()

In [ ]:
# Read in the data containing state centroid coordinates (latitudes and longitudes)

In [22]:
#EPA Air Quality Section
url = f"https://aqs.epa.gov/data/api/sampleData/byCounty?email=buhler8819@gmail.com&key={b_epa_key}&param=88101&bdate=20160101&edate=20160114&state=27&county=053"

In [23]:
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "Data": [
        {
            "cbsa_code": "33460",
            "county": "Hennepin",
            "county_code": "053",
            "date_gmt": "2016-01-01",
            "date_local": "2016-01-01",
            "date_of_last_change": "2016-07-07",
            "datum": "NAD83",
            "detection_limit": 5.0,
            "latitude": 44.965242,
            "longitude": -93.254759,
            "method": "Met One BAM-1020 Mass Monitor w/VSCC - Beta Attenuation",
            "method_code": "170",
            "method_type": "FEM",
            "parameter": "PM2.5 - Local Conditions",
            "parameter_code": "88101",
            "poc": 3,
            "qualifier": null,
            "sample_duration": "1 HOUR",
            "sample_frequency": "HOURLY",
            "sample_measurement": 14.0,
            "site_number": "0962",
            "state": "Minnesota",
            "state_code": "27",
            "time_gmt": "06:00",
            "time_local": "00:00",
            "uncertai